# Simulation of stochastic Schrödinger trajectories

In this notebook we demonstrate how to use the `SSESimulator` module to
generate ensembles of stochastic quantum trajectories for a continuously
monitored qubit.  The simulator implements the Stratonovich quantum
trajectory equations described in Dressel *et al.* (2017) for a qubit
subjected to a drive about the $y$‑axis and homodyne detection of the $z$
Pauli operator【32376578886034†L170-L196】.  Each trajectory yields an estimate of the
arrow‑of‑time log‑likelihood ratio
.  We compare the resulting
distributions to the analytic Gaussian prediction given in Eq. (5) of
Dressel *et al.*【32376578886034†L299-L312】.

Below we simulate an ensemble of trajectories for four different
durations $T/	au in 0.02,0.2,1.18,2.0$, as in Fig. 2 of the paper, and
visualise the histograms of $ln R$ together with the theoretical mean
$3T/(2	au)$ and variance $2T/	au$.  Because the original figure used $2	imes10^6$
trajectories, here we use a smaller ensemble to keep the computation
lightweight but still observe convergence towards the expected Gaussian.
You can increase `n_traj` to improve the statistics.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sse_simulation import SSESimulator, compute_histogram

# Define simulation parameters
tau = 1.0
Omega = 4 * np.pi  # chosen so that 2π/Ω = 0.5 τ【32376578886034†L299-L312】
dt = 1e-3

# Create the simulator
sim = SSESimulator(tau=tau, Omega=Omega, dt=dt)

# Define durations (in units of τ) matching Fig. 2
durations = [0.02, 0.2, 1.18, 2.0]

# Number of trajectories per duration
n_traj = 5000  # increase for smoother histograms

# Container for results
results = {}
for T in durations:
    print(f
Simulating T= {T} τ with {n_traj} trajectories...
)    lnR_values = sim.simulate_ensemble(n_traj, T)
    results[T] = lnR_values


In [ ]:
# Plot histograms and compare to Gaussian predictions
fig, axes = plt.subplots(2, 2, figsize=(10, 8), sharex=False, sharey=False)

for ax, T in zip(axes.ravel(), durations):
    data = results[T]
    # compute histogram
    bins = 60
    bin_centers, counts = compute_histogram(data, bins=bins, density=True)
    ax.bar(bin_centers, counts, width=bin_centers[1] - bin_centers[0], color='C0', alpha=0.6, label='Simulation')
    
    # analytic Gaussian: mean and variance from Dressel et al. Eq. (5)【32376578886034†L299-L312】
    mu = 3 * T / (2 * tau)
    sigma2 = 2 * T / tau
    sigma = np.sqrt(sigma2)
    xs = np.linspace(min(data), max(data), 200)
    gaussian = (1 / (np.sqrt(2 * np.pi * sigma2))) * np.exp(-0.5 * ((xs - mu) / sigma)**2)
    ax.plot(xs, gaussian, 'r--', lw=2, label='Gaussian prediction')

    # vertical line at zero to indicate sign of ln R
    ax.axvline(0.0, color='k', linestyle=':', linewidth=1)
    ax.axvline(mu, color='r', linestyle='-', linewidth=1, label='Mean')

    ax.set_title(f'T = {T} τ')
    ax.set_xlabel('ln R')
    ax.set_ylabel('Probability density')
    ax.legend(loc='upper right', fontsize='small')

fig.suptitle('Histograms of ln R for different durations')
plt.tight_layout()
plt.show()

In [ ]:
# Compute and display mean and variance of ln R for each duration
for T in durations:
    data = results[T]
    mean_lnR = np.mean(data)
    var_lnR = np.var(data)
    print(f'Duration T={T} τ: mean ln R = {mean_lnR:.3f}, variance = {var_lnR:.3f}')
